In [ ]:
!git clone https://github.com/chenyuntc/simple-faster-rcnn-pytorch

Faster-RCNN github link
--

https://github.com/chenyuntc/simple-faster-rcnn-pytorch


In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCdevkit_08-Jun-2007.tar

In [ ]:
!tar xvf VOCtrainval_06-Nov-2007.tar
!tar xvf VOCtest_06-Nov-2007.tar
!tar xvf VOCdevkit_08-Jun-2007.tar

In [ ]:
!pip install visdom scikit-image tqdm fire ipdb matplotlib torchnet

In [ ]:
!nohup python -m visdom.server &

In [ ]:
!python /content/simple-faster-rcnn-pytorch/train.py train --env='fasterrcnn' --plot-every=100

# Transformers

https://arxiv.org/abs/1706.03762

Abstract
--

기존 번역 모델들은 복잡한 RNN, CNN기반의 인코더-디코더 구조를 가졌다. 가장 성능이 좋은 모델은 attention mechanism으로 인코더 디코더를 연결하는 구조였다. transformer는 CNN, RNN 모듈을 완전히 없애고, 오직 attention mechanism을 기반으로 한다. 실험적으로 transformer는 두 MT task(WMT2014 english-to-german, english-to-franch)에서 우수한 성능을 보였으며 다른 task에서도 보편적으로 좋은 성능을 보였다.

- 성능이 우월했으며, 병렬화 통해 train에 요구되는 시간을 획기적으로 줄였다. 기계번역 뿐만 아니라 English constituency parsing 등의 다른 task 에도 일반화가 잘됬다.



Introduction
--

RNN, 특히 LSTM과 GRU는 시퀀스모델링 및 언어모델링, 기계번역 등의 transduction 문제에서 state of the art 접근법으로 자리매김해왔었다. recurrent 한 언어모델들과 encoder-decoder 구조의 한계를 넓히기 위한 많은 노력들이 계속되고있다.

recurrent한 모델은 전형적으로 input과 output 시퀀스의 토큰 위치에 따라 계산을 분해하여 진행한다. 계산시점에서 각 step에 따라 위치를 정렬시키고 hidden state $h_t$ 를 $h_{t−1}$ 과 t step의 input으로부터 생성한다. 이 본질적으로 sequential한 특성은 훈련과정에서의 병렬화를 배제시키는데, 이런 메모리 제약에 의해 샘플간 배치화를 제한하여 더 긴 길이의 시퀀스를 처리할 수록 critical한 문제가 된다. 최근 연구들이 factorization trick들과 conditional computation을 이용해 모델의 퍼포먼스는 향상시키며 계산 효율성을 눈에 띄게 향상시켜왔지만, sequential computation의 근본적인 문제는 여전히 남아있다.

어텐션 메커니즘 은 input과 output 시퀀스에서 거리에 상관없는 의존성(dependency) 모델링을 가능하게 함으로써 다양한 task의 시퀀스모델링 및 transduction 모델 등에서 빠질 수 없는 part가 되어가고있다. 그러나 몇 가지 경우를 제외하고는 attention 메커니즘은 항상 recurrent network와 함께 사용된다.

이 연구에서는 input과 output간에 global한 dependency를 추출하기위해 recurrence에서 벗어나 전적으로 attention 메커니즘에만 의존하는 transformer 모델을 제안한다. 트랜스포머는 유의미한 병렬화를 가능하게 하고, 8개의 P100 GPU로 12시간 정도만 training하면, 번역의 품질에 있어서 SOTA에 도달할 수 있다

Background
--

시퀀셜한 계산의 감소에 대한 목표는 Extended Neural GPU, ByteNet, ConvS2S 연구의 기반을 형성했다.

이것들은 모든 input, output 포지션에서 병렬적으로 hidden representation을 계산하는 CNN을 basic한 building block으로 사용하였다. 이 모델들에서는 input과 output의 임의의 두 위치로부터의 신호를 관계시키기위해 필요한 계산 수가 그 위치들간 거리에 따라 증가하는데, ConvS2S에서는 선형적으로, ByteNet에서는 logarithmically하게 증가한다. 이는 거리가 멀어질 수록 dependency를 학습하는데 어려워지게 만든다. 반면 트랜스포머에서는 어떤 상수 번의 계산만이 요구된다. 비록 어텐션 가중 position을 평균냄으로써 유효 해상도의 감소라는 cost가 있지만, 이후 살펴볼 Multi-Head Attention으로 상쇄시킬 수 있다.

때때로 intra-attention(내부 어텐션)이라고도 불리는 self-attention은 한 시퀀스의 representation을 계산하기 위해 해당 단일 시퀀스 내에서 서로다른 위치에 있는 요소들을 관련시키는 메커니즘이다.
이 self-attention은 reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representation 등의 다양한 task에서 성공적으로 사용되어왔다.

배열된 시퀀스의 recurrent 네트워크 대신 재귀적인 attention mechanism에 기반한 End-to-end 메모리 네트워크는 단일 언어 Question answering task와 언어 모델링 task 에서 좋은 퍼포먼스를 보여왔다.

그러나, 트랜스포머는 input과 output의 representation을 계산하기 위해 RNN이나 합성곱을 사용하지 않고 오로지 self-attention에만 의존하는 최초의 변환모델이다.

Model Architecture
--

가장 경쟁력 있는 신경망 기반 시퀀스 변형모델들은 encoder-decoder 구조를 가지고있다.
여기서 encoder는 input시퀀스의 연속적인 representation인 $(x_1, {\cdots},x_{n})$ 을 다른 연속적인 representation들의 시퀀스 $z=(z_1,{\cdots},z_{n})$ 으로 매핑한다. 이 z를 가지고 decoder는 output 시퀀스 $(y_1,{\cdots},y_{m})$
 를 생성한다. (각 time step에서 하나의 element를.) 각 타임스텝에서 다음 심볼을 생성할 때, 모델은 이전에 생성된 심볼들을 추가적인 input으로써 사용하기 때문에 Auto-Regressive하다.

트랜스포머는 위와 같은 전반적인 encoder-decoder 구조를 사용하는데, self-attention과 point-wise fully connected layer들을 encoder와 decoder 각각에 쌓아올려 이용한다.

<img src ='https://images.velog.io/images/changdaeoh/post/45b80d7d-c6ef-4746-aa6c-7a96a3012a6d/image.png'>

Encoder & decoder
--

- Encoder

N = 6개의 동일한 layer를 쌓아올려 구성되었다. 각각의 layer들은 2개의 sub-layer들로 구성되는데, 첫 번째는 multi-hear self-attention 메커니즘이고 두 번째는 단순 position-wise FC feed-forward 네트워크다.

이 각각의 sub-layer마다 residual connection을 차용하였고, 이후 layer normalization이 뒤따르도록 구성하였다. 즉, 각각의 sub-layer의 output이 LayerNorm( x + Sublayer(x) )가 되도록 하였다. 이 residual connection의 적용에 편의를 위해 임베딩 층을 포함한 모든 sub-layer들이 output 차원으로 $d_{model}$
​
 =512를 갖도록 통일했다. (residual connection시 element-wise addition을 진행하려면 차원이 같아야됨)

- Decoder
: decoder 또한 N = 6개의 동일한 layer를 쌓아올려 구성되었는데, encoder와 동일한 두 sub-layer 사이에 encoder의 output에 대해 multi-head attention을 수행하는 sub-layer를 추가로 삽입하였다.
encoder와 비슷하게 각 sub-layer 마다 layer normalization이 뒤따르는 residual connection을 사용하였으며, decoder stack 내의 self-attention layer를 수정하였는데, 어떤 i번째 위치에서 prediction을 진행할 때, 미래의 위치들에 접근하는 것을 불가능하게하고 해당 위치 i와 그 이전의 위치들에 대해서만 의존하도록 masking 기법을 이용하였다.

Attention
--

어텐션 함수는 query, key, value, output들이 모두 벡터일 때, query와 key-value 쌍의 집합을 output에 mapping하는 것이라고 설명될 수 있다. query와 그에 대응되는 key의 연관성 함수(compatibility function)에 의해 계산된 어텐션 가중치들로 value들을 weighted sum하여 어텐션 output이 계산된다.

<img src = 'https://media.vlpt.us/images/changdaeoh/post/eef7ae6b-872f-44f6-bfdd-bd6e4d34b357/image.png'>

Scaled Dot-Product Attention
--

"Scaled Dot-Product Attention"의 input은 $d_k$차원의 query와 key 그리고 $d_v$
​
 차원의 value 벡터들로 구성된다. 모든 query와 key를 내적(dot product) 하고 각각을 $d_k$ 로 나눈 뒤 value들에 대한 weight를 얻기 위해 softmax 함수를 적용한다.

실제로는 모든 쿼리들이 하나의 행렬 Q로써, 키와 밸류들은 행렬 K와 V로써 packed 되어 동시에(simultaneously) 어텐션 함수가 계산된다. 어텐션 output 또한 V와 shape가 동일한 행렬이 되며 아래와 같이 계산된다.

<img src = 'https://media.vlpt.us/images/changdaeoh/post/38f74618-6275-4b7e-92d3-a4908de4a52d/image.png'>

흔히 사용되는 어텐션 함수는 additive attention과 dot-product attention인데, dot-product attention의 경우 scaling factor로써의  ${\dfrac {1}{\sqrt{d_k}}}$
​
 를 제외하면 본 연구의 제안 알고리즘과 동일하다.
additive attention은 단일 은닉층의 feed-forward network를 이용해 연관성 함수를 계산한다. 두 방법이 이론상 동일한 복잡도를 갖는 반면, 실무에서는 dot-product attention이 고도로 최적화된 행렬곱 code로 구현될 수 있어 더 빠르고 더 공간효율적이다.

쿼리와 키 벡터의 차원 $d_k$
​
 가 작은 값일 떄는 additive attention과 dot-product attention이 유사한 퍼포먼스를 보이는 반면, $d_k$
​
 가 클 때 scaling을 적용하지 않으면 additive attention이 dot-product attention의 성능을 앞질렀다.
 

연구진들은 $d_k$
​
 가 커질 경우 dot product의 결과값도 커지는 경향이 있어, 결과적으로 softmax 함수에서 gradient가 매우 작아지는 saturate한 부분으로 $QK^T$
 가 input 된다고 의심하여, 이것을 방지하기 위해 ${\dfrac {1}{\sqrt{d_k}}}$
를 곱하여 스케일을 조정하였다.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
%matplotlib inline
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.9.0+cu102].
device:[cuda:0].


Scaled Dot-Product Attention (SDPA)
--

- Data $X \in \mathbb{R}^{n \times d}$ where $n$ is the number data and $d$ is the data dimension
- Query and Key $Q, K \in \mathbb{R}^{n \times d_K}$ 
- Value $V \in \mathbb{R}^{n \times d_V} $

$\text{Attention}(Q,K,V) = \text{softmax} \left( \frac{QK^T}{\sqrt{d_K}} \right)V \in \mathbb{R}^{n \times d_V} $

In [8]:
class ScaleDotPreoductAttention(nn.Module):
  def forward(self,Q,K,V,mask = None):
    d_k = K.size()[-1] # key dimension
    scores = Q.matmul(K.transpose(-2,-1)) / np.sqrt(d_K)
    if mask is not None:
      scores = scores.masked_fill(mask==0, -1e9)
    attention = F.softmax(scores,dim = -1)
    out = attention.matmul(V)
    return out,attention

#Demo run of scaled dot product attention
SPDA = ScaleDotPreoductAttention()
n_batch,d_K,d_V = 3,128,256 # d_K(=d_Q) does not necessarily be equal to d_V
n_Q,n_K,n_V = 30,50,50
Q = torch.rand(n_batch,n_Q,d_K)
K = torch.rand(n_batch,n_K,d_K)
V = torch.rand(n_batch,n_V,d_V)
out,attention = SPDA.forward(Q,K,V,mask = None)
def sh(x):
  return str(x.shape)[11:-1]
print('SDPA : Q%s K%s V%s => out%s attention %s' %(sh(Q),sh(K),sh(V),sh(out),sh(attention)))

## it supports 'multi-headed attention
n_batch,n_head,d_K,d_V = 3,5,128,256 
n_Q,n_K,n_V = 30,50,50 #n_K and n_V should be the same
Q = torch.rand(n_batch,n_head,n_Q,d_K)
K = torch.rand(n_batch,n_head,n_K,d_K)
V = torch.rand(n_batch,n_head,n_V,d_V)
out,attention = SPDA.forward(Q,K,V,mask = None)
#out : [n_batch X n_head X n_Q X d_V]
# attention : [n_batch X n_head X n_Q X n_K]
def sh(x):
  return str(x.shape)[11:-1]
print('SDPA : Q%s K%s V%s => out%s attention %s' %(sh(Q),sh(K),sh(V),sh(out),sh(attention)))

SDPA : Q[3, 30, 128] K[3, 50, 128] V[3, 50, 256] => out[3, 30, 256] attention [3, 30, 50]
SDPA : Q[3, 5, 30, 128] K[3, 5, 50, 128] V[3, 5, 50, 256] => out[3, 5, 30, 256] attention [3, 5, 30, 50]


Multi-Head Attention
--

$d_model$
​
 차원의 key, value, query들을 가지고 단일 어텐션을 수행하는 대신에 쿼리, 키, 벨류를 각각 $d_k$,$d_v$,$d_q$
​
  차원으로 변환하는 서로다른 h개의 학습가능한 선형 사영(linear projection)을 진행한 뒤, 각각의 다른 projected version에 대해 병렬적으로 어텐션을 진행하는 것이 더 이롭다는 사실을 확인하였다. 각 사영 버전들로부터 서로다른 h개의 $d_v$
​
 차원 결과들을 얻고 그 결과들을 concat 한 뒤 한번 더 linear projection하여 최종 결과벡터를 얻는다.

멀티헤드 어텐션은 모델이 서로다른 domain들의 서로다른 representation 부분공간(subspace)들로부터 결합적으로(jointly) 정보에 접근하도록 한다. 단일 어텐션 헤드에서는, averaging 때문에 이러한 부분공간들로부터의 joint한 정보접근이 억제된다.

<img src= 'https://media.vlpt.us/images/changdaeoh/post/fbbb9832-272d-40e2-9a10-628c5e9c8def/image.png'>

- Multi-Headed Attention (MHA)

$\text{head}_{\color{red}i} = \text{Attention}(Q {\color{green}W}^Q_{\color{red}i},K {\color{green}W}^K_{\color{red}i}, V {\color{green}W}^V_{\color{red}i}) $

In [9]:
class MultiHeadedAttention(nn.Module):
    def __init__(self,d_feat=128,n_head=5,actv=F.relu,USE_BIAS=True,dropout_p=0.1,device=None):
        """
        :param d_feat: feature dimension
        :param n_head: number of heads
        :param actv: activation after each linear layer
        :param USE_BIAS: whether to use bias
        :param dropout_p: dropout rate
        :device: which device to use (e.g., cuda:0)
        """
        super(MultiHeadedAttention,self).__init__()
        if (d_feat%n_head) != 0:
            raise ValueError("d_feat(%d) should be divisible by b_head(%d)"%(d_feat,n_head)) 
        self.d_feat = d_feat
        self.n_head = n_head
        self.d_head = self.d_feat // self.n_head
        self.actv = actv
        self.USE_BIAS = USE_BIAS
        self.dropout_p = dropout_p # prob. of zeroed

        self.lin_Q = nn.Linear(self.d_feat,self.d_feat,self.USE_BIAS)
        self.lin_K = nn.Linear(self.d_feat,self.d_feat,self.USE_BIAS)
        self.lin_V = nn.Linear(self.d_feat,self.d_feat,self.USE_BIAS)
        self.lin_O = nn.Linear(self.d_feat,self.d_feat,self.USE_BIAS)
        # 행렬곱 때문에 펼쳐줘야하기 때문에 Linear하게 펼쳐 줌

        self.dropout = nn.Dropout(p=self.dropout_p)
    
    def forward(self,Q,K,V,mask=None):
        """
        :param Q: [n_batch, n_Q, d_feat]
        :param K: [n_batch, n_K, d_feat]
        :param V: [n_batch, n_V, d_feat] <= n_K and n_V must be the same 
        :param mask: 
        """
        n_batch = Q.shape[0]
        Q_feat = self.lin_Q(Q) 
        K_feat = self.lin_K(K) 
        V_feat = self.lin_V(V)
        # Q_feat: [n_batch, n_Q, d_feat]
        # K_feat: [n_batch, n_K, d_feat]
        # V_feat: [n_batch, n_V, d_feat]

        # Multi-head split of Q, K, and V (d_feat = n_head*d_head)
        Q_split = Q_feat.view(n_batch, -1, self.n_head, self.d_head).permute(0, 2, 1, 3)
        K_split = K_feat.view(n_batch, -1, self.n_head, self.d_head).permute(0, 2, 1, 3)
        V_split = V_feat.view(n_batch, -1, self.n_head, self.d_head).permute(0, 2, 1, 3)
        # Q_split: [n_batch, n_head, n_Q, d_head]
        # K_split: [n_batch, n_head, n_K, d_head]
        # V_split: [n_batch, n_head, n_V, d_head]

        # Multi-Headed Attention
        d_K = K.size()[-1] # key dimension
        scores = torch.matmul(Q_split, K_split.permute(0,1,3,2))
        if mask is not None:
            scores = scores.masked_fill(mask==0,-1e9)
        attention = torch.softmax(scores,dim=-1)
        x_raw = torch.matmul(self.dropout(attention),V_split) # dropout is NOT mentioned in the paper
        # attention: [n_batch, n_head, n_Q, n_K]
        # x_raw: [n_batch, n_head, n_Q, d_head]

        # Reshape x
        x_rsh1 = x_raw.permute(0,2,1,3).contiguous()
        # x_rsh1: [n_batch, n_Q, n_head, d_head]
        x_rsh2 = x_rsh1.view(n_batch,-1,self.d_feat)
        # x_rsh2: [n_batch, n_Q, d_feat]

        # Linear
        x = self.lin_O(x_rsh2)
        # x: [n_batch, n_Q, d_feat]
        out = {'Q_feat':Q_feat,'K_feat':K_feat,'V_feat':V_feat,
               'Q_split':Q_split,'K_split':K_split,'V_split':V_split,
               'scores':scores,'attention':attention,
               'x_raw':x_raw,'x_rsh1':x_rsh1,'x_rsh2':x_rsh2,'x':x}
        return out

# Self-Attention Layer
n_batch = 128
n_src   = 32
d_feat  = 200
n_head  = 5
src = torch.rand(n_batch,n_src,d_feat)
self_attention = MultiHeadedAttention(
    d_feat=d_feat,n_head=n_head,actv=F.relu,USE_BIAS=True,dropout_p=0.1,device=device)
out = self_attention.forward(src,src,src,mask=None)

Q_feat,K_feat,V_feat = out['Q_feat'],out['K_feat'],out['V_feat']
Q_split,K_split,V_split = out['Q_split'],out['K_split'],out['V_split']
scores,attention = out['scores'],out['attention']
x_raw,x_rsh1,x_rsh2,x = out['x_raw'],out['x_rsh1'],out['x_rsh2'],out['x']

# Print out shapes
def sh(_x): return str(_x.shape)[11:-1] 
print ("Input src:\t%s  \t= [n_batch, n_src, d_feat]"%(sh(src)))
print ()
print ("Q_feat:   \t%s  \t= [n_batch, n_src, d_feat]"%(sh(Q_feat)))
print ("K_feat:   \t%s  \t= [n_batch, n_src, d_feat]"%(sh(K_feat)))
print ("V_feat:   \t%s  \t= [n_batch, n_src, d_feat]"%(sh(V_feat)))
print ()
print ("Q_split:  \t%s  \t= [n_batch, n_head, n_src, d_head]"%(sh(Q_split)))
print ("K_split:  \t%s  \t= [n_batch, n_head, n_src, d_head]"%(sh(K_split)))
print ("V_split:  \t%s  \t= [n_batch, n_head, n_src, d_head]"%(sh(V_split)))
print ()
print ("scores:   \t%s  \t= [n_batch, n_head, n_src, n_src]"%(sh(scores)))
print ("attention:\t%s  \t= [n_batch, n_head, n_src, n_src]"%(sh(attention)))
print ()
print ("x_raw:    \t%s  \t= [n_batch, n_head, n_src, d_head]"%(sh(x_raw)))
print ("x_rsh1:   \t%s  \t= [n_batch, n_src, n_head, d_head]"%(sh(x_rsh1)))
print ("x_rsh2:   \t%s  \t= [n_batch, n_src, d_feat]"%(sh(x_rsh2)))
print ()
print ("Output x: \t%s  \t= [n_batch, n_src, d_feat]"%(sh(x)))

Input src:	[128, 32, 200]  	= [n_batch, n_src, d_feat]

Q_feat:   	[128, 32, 200]  	= [n_batch, n_src, d_feat]
K_feat:   	[128, 32, 200]  	= [n_batch, n_src, d_feat]
V_feat:   	[128, 32, 200]  	= [n_batch, n_src, d_feat]

Q_split:  	[128, 5, 32, 40]  	= [n_batch, n_head, n_src, d_head]
K_split:  	[128, 5, 32, 40]  	= [n_batch, n_head, n_src, d_head]
V_split:  	[128, 5, 32, 40]  	= [n_batch, n_head, n_src, d_head]

scores:   	[128, 5, 32, 32]  	= [n_batch, n_head, n_src, n_src]
attention:	[128, 5, 32, 32]  	= [n_batch, n_head, n_src, n_src]

x_raw:    	[128, 5, 32, 40]  	= [n_batch, n_head, n_src, d_head]
x_rsh1:   	[128, 32, 5, 40]  	= [n_batch, n_src, n_head, d_head]
x_rsh2:   	[128, 32, 200]  	= [n_batch, n_src, d_feat]

Output x: 	[128, 32, 200]  	= [n_batch, n_src, d_feat]


Multi-Head Attention
--

$d_{model}$
​
 차원의 key, value, query들을 가지고 단일 어텐션을 수행하는 대신에 쿼리, 키, 벨류를 각각 $d_k$,$d_v$,$d_q$
​
  차원으로 변환하는 서로다른 h개의 학습가능한 선형 사영(linear projection)을 진행한 뒤, 각각의 다른 projected version에 대해 병렬적으로 어텐션을 진행하는 것이 더 이롭다는 사실을 확인하였다. 각 사영 버전들로부터 서로다른 h개의 $d_v$
​
 차원 결과들을 얻고 그 결과들을 concat 한 뒤 한번 더 linear projection하여 최종 결과벡터를 얻는다.

 
멀티헤드 어텐션은 모델이 서로다른 domain들의 서로다른 representation 부분공간(subspace)들로부터 결합적으로(jointly) 정보에 접근하도록 한다. 단일 어텐션 헤드에서는, averaging 때문에 이러한 부분공간들로부터의 joint한 정보접근이 억제된다.

<img src= 'https://media.vlpt.us/images/changdaeoh/post/fbbb9832-272d-40e2-9a10-628c5e9c8def/image.png'>

# Applications of Attention in our Model

트랜스포머는 multi-head attention을 3가지 다른 방식으로 사용한다.

- 디코더의 "encoder-decoder attention" 층에 input되는 쿼리들은 이전 decoder layer의 output으로부터 오고, 키와 벨류들은 encoder의 최종 output으로부터 온다. 이것은 decoder의 모든 위치에서 input 시퀀스의 모든 position에 접근할 수 있음을 허용한다. 이것은 seq2seq 모델의 전형적인 encoder-decoder attention을 모방한 어텐션이다.

<img src ='https://images.velog.io/images/changdaeoh/post/45b80d7d-c6ef-4746-aa6c-7a96a3012a6d/image.png'>

인코더는 self-attention 레이어를 포함한다. self-attention 레이어에 input되는 모든 키, 벨류, 쿼리들은 동일한 하나의 시퀀스(이전 인코더 층의 output)로부터 온다. 각 인코딩 층의 position들은 이전 인코딩 층의 모든 position에 접근할 수 있다.

 Position-wise Feed-Forward Networks
 --

어텐션 sub-layer에 더하여, 우리의 encoder와 decoder는 각각의 position에 대해 독립적으로, 동등하게 적용되는 fully connected feed-forward network를 포함한다. 이 층은 두 번의 선형변환(linear transformation)과 그 사이에 ReLU activation을 포함한다. 공식은 아래와 같다.

<img src = 'https://media.vlpt.us/images/changdaeoh/post/b6978218-6ce7-4c17-bfee-b6384bd438fc/image.png'>

선형변환이 다른 position들에 대해 동일하게 적용되는 반면, 층간(layer to layer)에는 서로 다른 파라미터($W_1,W_2$) 를 사용한다. 이는 커널사이즈가 1인 convolution을 두번 진행하는 것으로도 생각할 수 있다. input과 output의 차원은 동일하게 $d_{model}$=512 이고, FFN 내부의 hidden layer는 2048의 차원을 갖는다.

Embeddings and Softmax
--

다른 시퀀스 변환 모델들과 마찬가지로 input과 output 토큰들을 $d_{model}$
​차원 벡터로 변환하기 위해 학습가능한 임베딩을 사용하였다. 또한, decoder의 output으로 next-token 확률을 예측하기위해 학습가능한 선형변환과 소프트맥스 함수를 사용하였다. 이때, 두 임베딩 층과 softmax 이전의 선형변환에 동일한 가중치행렬을 공유하여 사용하였다. 임베딩 층들에서는 그 가중치들에 ${\sqrt{d_{model}}}$ 를 곱하여 사용하였다.

 Positional Encoding
 --

저자들이 제안하는 모델이 어떠한 recurrence나 convolution도 포함하지 않기 때문에, 모델이 시퀀스의 순서를 반영 및 사용할 수 있도록 시퀀스 내의 토큰들의 상대적/절대적 위치에 관한 정보들을 주입해야만 했다. 이를 위해 우리는 encoder와 decoder의 최 하단에 위치한 input embedding에 "positional encodings"를 더하였다. positional encoding은 임베딩과 마찬가지로 $d_{model}$
​
  차원을 갖게하였고 그렇게해서 두 벡터가 더해질 수 있게하였다. 학습가능한, 혹은 고정된 positional encoding의 선택지는 매우 다양하다.

이 연구에서는 다른 주기의 sine, cosine 함수들을 사용하였다.

<img src = 'https://media.vlpt.us/images/changdaeoh/post/22a8d912-d34b-4977-9c69-2b237f6f3735/image.png'>

( pos : 토큰의 위치 인덱스, i : 벡터의 차원 인덱스)

<img src = 'https://images.velog.io/images/changdaeoh/post/bf874511-fc7e-4c63-8f69-3f55cb9d7eac/image.png'>

각 위치 인코딩의 차원 i들은 사인 곡선에 대응된다. 이러한 함수를 선택한 이유는 그 어떠한 고정된 offset값 k에 대해서도 $PE_{pos+k}$가 $PE_{pos}$
의 선형함수로써 표현될 수 있기에 모델이 상대적인 위치를 쉽게 학습할 것이라 가설을 세웠기 때문이다.

연구자들이 sine, cosine 기반 encoding 대신 학습가능한 위치 임베딩을 사용해보았는데, 두 방식이 거의 동일한 결과를 냈다. 우리는 결국 sinusoidal version을 선택했는데, 이 방식의 경우 모델이 훈련되는 동안 보았던 길이보다 더 긴 시퀀스에 대해서도 얼마든지 extrapolate 할 수 있기 때문이다.

transformer의 장점
--

attention distribution이 시각화가 가능하여 사용하는 모델의 해석 가능성(설명력)을 높여준다

# 2021 08 23 마지막 수업


In [10]:
from __future__ import unicode_literals, print_function, division
# 이 from은 사용자가 편리하게 하게 만든 함수들
# unicode -> 아스키 코드를 변환하는 함수
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

- 데이터 다운로드(https://download.pytorch.org/tutorial/data.zip)

In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip  #wget: we get의 약자. -> 리눅스에서 허용. 윈도우는 wget을 설치해야 됨.

In [ ]:
!unzip /content/data.zip

문자 단위 RNN 튜토리얼에서 사용된 문자 인코딩과 유사하게, 언어의 각 단어들을 One-Hot 벡터 또는 그 단어의 주소에만 단 하나의 1을 제외. 모두 0인 큰 벡터로 표현합니다. 한 가지 언어에 있는 수십 개의 문자와 달리 번역에는 아주 많은 단어들이 있기 때문에 인코딩 벡터는 더 큽니다.

Word Index
--

나중에 네트워크의 입력 및 목표로 사용하려면 단어 당 고유 번호가 필요합니다. 이것을 추적하기 위해 아래와 같이 나눕니다.


In [ ]:
SOS_token = 0 
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name=name
        self.word2index = {}
        self.word2.count = {}
        self.index2word={0:"SOS", 1:"EOS"}
        self.n_words=2 # SOS와 EOS 포함

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWorld()

데이터 유니코드 -> ASCII
--

데이터 파일이 모두 유니 코드로 되어있어 간단하게 하기 위해 유니 코드 문자를 ASCII로 변환하고, 모든 문자를 소문자로 만들고, 대부분의 구성을 지워줍니다.


In [16]:
# 유니 코드 문자열을 일반 ASCII로 변환하십시오.
# https://stackoverflow.com/a/518232/2809427

def unicodeToAscii(s):
    return ''.join( c for c in unicodedata.normalize('NFD', s)
    if unicodedata.category(c) != "Mn")
    # 소문자, 다듬기, 그리고 문자가 아닌 문자 제거

def normalizeString(s):
    s=unicodeToAscii(s.lower().strip())
    s=re.sub(r"([.!?])", r"W1", s)
    s=re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

데이터 분리
--

데이터 파일을 읽기 위해 파일을 라인으로 나누고, 줄 안의 내용을 쌍으로 나눕니다.

데이터 셋 만들기

많은 예제 문장에 신속하게 학습하기를 원하기 때문에 비교적 짧고 간단한 문장으로만 데이터 셋을 만듭니다.

- 최대 10단어 (종료 문장 부호 포함)

- "I am 또는 He is" 등의 형태로 번역되는 문장으로 필터링

In [ ]:
MAX_LENGTH = 10

eng_prefixes=(
    "i am","i m",
    'he is','he s',
    'she is','she s',
    'you are','you re',
    'we are','we re',
    'they are','they re'
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
    len(p[1].split(' '))

데이터 준비를 위한 프리 프로세싱 과정

- 텍스트 파일을 읽고 줄로 분리하고, 줄을 쌍으로 분리합니다.

- 텍스트를 정규화하고 길이와 내용으로 필터링 합니다.

- 쌍을 이룬 문장들로 단어 리스트를 생성합니다.

In [ ]:
def prepareData(lang1, lang2, revrese=False):
    input_lang, output_lang, paris = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" %len(pairs))
    pairs=filterPairs(pairs)
    print("Trimmed to %s sentence pairsss"%len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words: ")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, out_put_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng','fra',True)

Seq2Seq 모델 생성

RNN은 시퀀스에서 작동하고 다음 단계의 입력으로 자신의 출력을 사용하는 네트워크 입니다.

In [18]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded=self.embedding(input).view(1,1,-1)
        output=embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

디코더
--

디코더는 인코더 출력 벡터를 받아서 번역을 생성하기 위한 시퀀스를 출력합니다.